# Divisão

O controle é divido em quatro arquivos, e abaixo apresentaremos a funcionalidade de cada um deles. O init.py não possui código extenso, então não precisará dessa explicação


# Recomendacao_controller.py

In [ ]:
from flask import Blueprint, request, jsonify
from Services.treinamento.recomendacao_instancia import recomendacao_service 

# Cria o blueprint
recomendacao_bp = Blueprint('recomendacao', __name__)

# Rotas
@recomendacao_bp.route('/recomendar', methods=['POST'])
def recomendarViagens():
    try:
        data = request.get_json() 
        user_id = data.get("id")

        if not user_id:
            return jsonify({"ok": False,"error":"usuario não identificado"}),401
        
        recomendacoes = recomendacao_service.recomendarParaUsuarioNovo(user_id)

        return jsonify({"ok":True,"recomendacoes":recomendacoes})
            
    
    except ValueError as e:
        return jsonify({"error": str(e)}), 400
    
    except Exception as e:
        print(f"erro encontrado: {e}")
        return jsonify({"ok": False,'error': f'Erro interno: {e}'}), 500
    

A função recomendarViagens() é responsável por processar as requisições de recomendação enviadas pelo front-end. Quando o usuário solicita sugestões de viagens, o front-end envia um POST para o endpoint /api/recomendacao/recomendar, contendo o ID do usuário no corpo da requisição.

O código começa lendo esses dados com request.get_json() e verificando se o identificador foi informado corretamente. Caso o user_id esteja ausente, o sistema retorna uma resposta de erro indicando que o usuário não foi identificado. Se o ID for válido, a função chama o método recomendacao_service.recomendarParaUsuarioNovo(user_id), que utiliza o modelo TF-IDF para calcular a similaridade entre o perfil do usuário e as características das viagens disponíveis.

Por fim, a rota retorna uma resposta JSON contendo a lista de recomendações geradas. O tratamento de exceções garante que, em caso de erro interno ou de dados inválidos, o sistema responda de forma segura e informativa ao usuário.

# User_controlle.py

In [ ]:
from flask import Blueprint, request, jsonify
from Services.usuario.user_service import UserService

# Cria o blueprint
user_bp = Blueprint('user', __name__)

# Instância do service
user_service = UserService()

# Rotas

@user_bp.route('/login', methods=['POST'])
def login():
    try:
        data = request.get_json()
        user = user_service.login(data['email'], data['senha'])
        return jsonify({"ok": True, "user": user, "message": "Login feito com sucesso!"})

    except ValueError as e:
        return jsonify({"ok": False, "error": str(e)}), 401
        
    except Exception as e:
        return jsonify({"ok": False,'error': f'Erro interno: {e}'}), 500

@user_bp.route("/cadastrar", methods=["POST"]) 
def cadastrar():
    try:
        data = request.get_json()
        user_id = user_service.cadastrar(data['nome'], data['email'], data['senha'])
        return jsonify({"ok": True,"id": user_id, "message": "Cadastrado com sucesso!"}), 201
        
    except ValueError as e:
        return jsonify({"ok": False,"ok": False,'error': str(e)}), 400
        
    except Exception as e:
        return jsonify({"ok": False,'error': f'Erro interno: {e}'}), 500

A função define o controlador responsável pelas rotas de login e cadastro de usuários no backend. Por meio do blueprint user_bp, o sistema organiza essas rotas sob o caminho /api/user.

Na rota /login, o backend recebe um POST com o e-mail e a senha do usuário, chamando o método user_service.login() para validar as credenciais e retornar os dados do usuário autenticado. Já na rota /cadastrar, os dados de nome, e-mail e senha são enviados e tratados pelo método user_service.cadastrar(), que cria um novo registro no banco.

Ambas as funções retornam respostas em formato JSON, informando sucesso ou erro, e utilizam blocos try/except para tratar exceções e garantir mensagens claras em caso de falhas.

# Viagem_controller.py

In [ ]:
from flask import Blueprint, request, jsonify
from Services.viagem.viagem_service import ViagemService

# Cria o blueprint
viagem_bp = Blueprint('viagem', __name__)

# Instância do service
viagem_service = ViagemService()

# Rotas

@viagem_bp.route('/preferences', methods=['POST'])
def cadastrar_preferencias():
    try:
        data = request.get_json()
        viagem_user = data.get("viagem_user")
        clima = data.get("clima")
        preco = data.get("preco")
        companhia = data.get("companhia")
        if not viagem_user:
            return jsonify({"ok": False,"error": "Parâmetros inválidos"}), 400
        print(viagem_user)
        
        viagem_service.cadastrarPreferencias(viagem_user, clima, preco, companhia)
        return jsonify({"ok": True, "message": "Preferências cadastradas"})
    
    except Exception as e:
        return jsonify({"ok": False, 'error': f'Erro interno: {e}'}), 500

@viagem_bp.route('/generos', methods=['POST'])
def cadastrar_generos():
    # genero ira retornar para a pagina de recomendacao: home
    try:
        data = request.get_json()
        viagem_user = data.get("UsuarioId")
        id_genero = data.get("GeneroId")
        preferencia = data.get("Preferencia")
        if not viagem_user or not id_genero:
            return jsonify({"ok": False,"error": "Parâmetros inválidos"}), 400
        
        viagem_service.cadastrarGeneros(viagem_user, id_genero, preferencia)
        return jsonify({"ok": True, "message": "Gêneros cadastrados"})
    
    except Exception as e:
        return jsonify({"ok": False,'error': f'Erro interno: {e}'}), 500

@viagem_bp.route('/lazeres', methods=['POST'])
def cadastrar_lazeres():
    try:
        data = request.get_json()
        viagem_user = data.get("UsuarioId")
        id_lazer = data.get("LazerId")
        intensidade = data.get("Intensidade")
        if not viagem_user or not id_lazer:
            return jsonify({"ok": False,"error": "Parâmetros inválidos"}), 400
        
        viagem_service.cadastrarLazeres(viagem_user, id_lazer, intensidade)
        return jsonify({"ok": True, "message": "Lazeres cadastrados"}),200
    
    except Exception as e:
        return jsonify({"ok": False,'error': f'Erro interno: {e}'}), 500
    
@viagem_bp.route("/comprar",methods=["POST"])
def comprarViagem():
    try:
        data = request.get_json()
        if (not data):
            return jsonify({"ok":False,
                            "error": "Parametros invalidos"}),400
        viagem_id = data.get("viagemId")
        usuario_id = data.get("userId")
        pontuacao = 0

        ja_comprou = viagem_service.checarCompra(viagem_id,usuario_id)
        if (ja_comprou):
            return jsonify({"ok":False,"message":"Usuario já comprou esta viagem"}),400

        viagem_service.comprar(viagem_id,usuario_id,pontuacao)
        return jsonify({"ok": True, "message": "Compra feita com sucesso"}),200
    
    except Exception as e:
        return jsonify({"ok":False,"error":f"Erro interno:{e}"}),500
    
@viagem_bp.route("/avaliar",methods=["POST"])
def avaliarViagem():
    try:
        data = request.get_json()
        if (not data):
            return jsonify({"ok":False,
                            "error": "Parametros invalidos"}),400
        viagem_id = data.get("viagemId")
        usuario_id = data.get("userId")
        pontuacao = data.get("pontuacao")
        viagem_service.avaliar(viagem_id,usuario_id,pontuacao)
        return jsonify({"ok": True, "message": "viagem avaliada com sucesso"}),200
    except Exception as e:
        return jsonify({"ok":False,"error":f"Erro interno:{e}"}),500
    
@viagem_bp.route("/obter",methods=["POST"])
def obterViagens():
    try:
        data = request.get_json()
        if (not data):
            return jsonify({"ok":False,
                            "error": "Parametros invalidos"}),400
        usuario_id = data.get("userId")

        viagens = viagem_service.getViagemUsuario(usuario_id)
        return jsonify({"ok": True, "message": "Viagens obtidas com sucesso", "viagens": viagens}),200
    
    except Exception as e:
        return jsonify({"ok":False,"error":f"Erro interno:{e}"}),500

Esse trecho de código define o controlador responsável pelas rotas de viagens do sistema, permitindo que o backend receba e gerencie informações relacionadas às preferências, gêneros, lazeres, compras e avaliações dos usuários, por meio do blueprint viagem_bp, todas as rotas ficam organizadas sob o caminho /api/viagem.
Cada função dentro do controlador trata um tipo específico de requisição:

A rota /preferences registra as preferências gerais do usuário (clima, preço e companhia desejada).
As rotas /generos e /lazeres recebem as escolhas e intensidades de gêneros e lazeres associados ao perfil de viagem.
A rota /comprar processa uma compra de viagem, verificando se o usuário já adquiriu o destino antes de registrar a transação.
A rota /avaliar permite que o usuário avalie uma viagem após a compra.
Por fim, a rota /obter recupera as viagens já associadas ao usuário.

Cada rota utiliza request.get_json() para ler os dados enviados pelo front-end e chama funções do ViagemService, que executam a lógica no banco de dados. As respostas são enviadas em formato JSON, com mensagens de sucesso ou erro, garantindo que o sistema seja robusto e seguro em caso de falhas durante o processamento.